###TASK 5: TITANIC KAGGLE CHALLENGE
The following code is an attempt to try to answer the kaggle challenge from Titanic dataset on: https://www.kaggle.com/c/titanic/overview/evaluation.

FIRST, LETS IMPORT WGET

In [ ]:
!pip install wget

NOW, LET'S USE IT TO DOWNLOAD THE DATASET!

Unfortunately, after a few days of struggle, I couldnt find a way to solely download the dataset from kaggle. So please download it from the webpage and import it to your google drive, doin the necessary changes on directories.

In [ ]:
import wget, os, zipfile
from zipfile import ZipFile

from google.colab import drive
drive.mount("/content/drive/")
#!ls "/content/drive/My Drive/CURSO_Redes_e_IA_Clécio/TITANIC_CHALLENGE/"
if os.path.exists('/content/drive/My Drive/CURSO_Redes_e_IA_Clécio/TITANIC_CHALLENGE/gender_submission.csv'):
    print(' ** Dataset already downloaded!')
    print(' ** Your dataset is ready to rock.')
else:
    print(' ** Dataset not found. Please download it and import into your drive.')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
 ** Dataset already downloaded!
 ** Your dataset is ready to rock.


NOW IT'S TIME TO DO PREPROCESSING

In [ ]:
import pandas as pd
import numpy as np

gender = pd.read_csv('/content/drive/My Drive/CURSO_Redes_e_IA_Clécio/TITANIC_CHALLENGE/gender_submission.csv', header=0)
test = pd.read_csv('/content/drive/My Drive/CURSO_Redes_e_IA_Clécio/TITANIC_CHALLENGE/test.csv', header=0)
train = pd.read_csv('/content/drive/My Drive/CURSO_Redes_e_IA_Clécio/TITANIC_CHALLENGE/train.csv', header=0)

print(' ** Training set shape is:', train.shape,
      '\n ** It has the following data types:', train.columns,
      '\n ** Test set is:', test.shape,
      '\n ** Gender set is:', gender.shape,
      '\n ** This dataset has the following columns:', train.columns,
      "\n ** You may notice they're full of problems. Let's check them out.")

print("\n ** First, the labels column is inside the train data.\
Let's separate training data from labels.")
labels = train['Survived']
print(" ** We'll be also saving the victim's names.")
train_victim_names = train['Name']
test_victim_names = test['Name']
del train['Survived'] 
print(' ** Training set new shape is:', train.shape)

print(" ** Let's check for missing data...")
print(" ** Well, there is some missing data on Age column. Let's replace it \
with median. \n ** There's on 'Embarked' column as well, so we're gonna replace\
for 'S', as it's the string with most ocurrencies.")

print(" ** Ticket, Name and Cabin column doesn't seem very helpful. Let's remove it.")
del train['Ticket'], test['Ticket'], train['Name'], test['Name'], \
    train['Cabin'], test['Cabin'], train['PassengerId'], test['PassengerId']

medians = train['Age'].median()
mediansf = train['Fare'].median()

train['Age'] = train['Age'].replace(to_replace=np.NaN,value=medians)
test['Age'] = test['Age'].replace(to_replace=np.NaN,value=medians)
train['Fare'] = train['Fare'].replace(to_replace=np.NaN,value=mediansf)
test['Fare'] = test['Fare'].replace(to_replace=np.NaN,value=mediansf)
train['Embarked'] = train['Embarked'].replace(to_replace=np.NaN,value='S')
test['Embarked'] = test['Embarked'].replace(to_replace=np.NaN,value='S')
from collections import Counter
cn = Counter(train['Embarked'])
###FIZEMOS ISSO PORQUE NÃO HÁ GARANTIA DE QUE NO GRUPO DE TESTE NÃO SERIAM
##NECESSÁRIAS INCLUIR NOVAS CLASSES, QUE NUNCA APARECERAM ANTES.
print(" ** Done!")

 ** Training set shape is: (891, 12) 
 ** It has the following data types: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object') 
 ** Test set is: (418, 11) 
 ** Gender set is: (418, 2) 
 ** This dataset has the following columns: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object') 
 ** You may notice they're full of problems. Let's check them out.

 ** First, the labels column is inside the train data.Let's separate training data from labels.
 ** We'll be also saving the victim's names.
 ** Training set new shape is: (891, 11)
 ** Let's check for missing data...
 ** Well, there is some missing data on Age column. Let's replace it with median. 
 ** There's on 'Embarked' column as well, so we're gonna replacefor 'S', as it's the string with most ocurrencies.
 ** Ticket, Name and Cabin colum

CONTINUE PREPROCESSING, WITH FINAL STEPS

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer 

print(" ** It's time to convert these strings into values.")
LE = LabelEncoder()
allcolumns = train.columns
columns = ['Sex', 'Embarked']

#for i in columns:
train['Sex'] = LE.fit_transform(train['Sex'])
test['Sex'] = LE.transform(test['Sex'])
train['Embarked'] = LE.fit_transform(train['Embarked'])
test['Embarked'] = LE.transform(test['Embarked'])


print(" ** Next action: Let's normalyze those values!")

SS = StandardScaler()
train = SS.fit_transform(train)
test = SS.transform(test)

num_classes = train.shape[1]
print(" ** Here we have a training set with %s classes." % num_classes)


 ** It's time to convert these strings into values.
 ** Next action: Let's normalyze those values!
 ** Here we have a training set with 7 classes.


NOW, LET'S TRAIN SOME CLASSIFIERS AND EVALUATE THEM! WHICH OF THESE SHOULD DO BEST?  
Feel free to check their correlation graphs.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_1 = RandomForestClassifier()
model_1.fit(train, labels)
survivors = model_1.predict(test)
acc = round(model_1.score(train, labels)*100, 2)
print(' ** Accuracy from Random Forest Classifier: {} %'. format(acc))

from sklearn import svm
model_2 = svm.SVC()
model_2.fit(train, labels)
acc = round(model_2.score(train, labels)*100, 2)
print(' ** Accuracy from Support Vector Machine Classifier: {} %'. format(acc))

from sklearn.naive_bayes import GaussianNB
model_3 = GaussianNB()
model_3.fit(train, labels)
acc = round(model_3.score(train, labels)*100, 2)
print(' ** Accuracy from Gaussian Naive-Bayes: {} %'. format(acc))

from sklearn.neighbors import KNeighborsClassifier
model_4 = KNeighborsClassifier(n_neighbors = 3)
model_4.fit(train, labels)
acc = round(model_4.score(train, labels)*100, 2)
print(' ** Accuracy from K Neighbours Classifier: {} %'. format(acc))

from sklearn.tree import DecisionTreeClassifier
model_5 = DecisionTreeClassifier()
model_5.fit(train, labels)
acc = round(model_5.score(train, labels)*100, 2)
print(' ** Accuracy from Decision Tree Classifier: {} %'. format(acc))

 ** Accuracy from Random Forest Classifier: 97.87 %
 ** Accuracy from Support Vector Machine Classifier: 83.95 %
 ** Accuracy from Gaussian Naive-Bayes: 79.24 %
 ** Accuracy from K Neighbours Classifier: 87.43 %
 ** Accuracy from Decision Tree Classifier: 97.98 %


Let's use the best classifier to now check out who survived.  
One minute of silence now, as we check the 
obituary.

In [ ]:
from tabulate import tabulate

survived = np.empty(shape=(0,0))
for x in range(len(survivors)):
    if survivors[x] == 0:
        result = 'No'
    else:
        result = 'Yes'
    survived = np.append(survived, result)
print(survived.shape)
    
table = [(test_victim_names, survived)]
print(tabulate(table, headers=["Name", "Survived?"]))

(418,)
Name                                                 Survived?
---------------------------------------------------  --------------------------------------------------------------------------
0                                  Kelly, Mr. James  ['No' 'No' 'No' 'Yes' 'No' 'No' 'No' 'No' 'Yes' 'No' 'No' 'No' 'Yes' 'No'
1                  Wilkes, Mrs. James (Ellen Needs)   'Yes' 'Yes' 'No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'Yes'
2                         Myles, Mr. Thomas Francis   'Yes' 'Yes' 'No' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No' 'No' 'No'
3                                  Wirz, Mr. Albert   'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No' 'No' 'Yes' 'No' 'No' 'No' 'Yes'
4      Hirvonen, Mrs. Alexander (Helga E Lindqvist)   'Yes' 'No' 'No' 'No' 'No' 'No' 'Yes' 'No' 'No' 'No' 'Yes' 'Yes' 'Yes'
                           ...                        'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'No' 'No' 'Yes' 'No' 'No' 'Yes' 'No'
413                              Spector, Mr. Woolf